Berner Fachhochschule BFH - MAS Data Science - Graph Machine Learning - Master Thesis FS/2022 Thomas Iten

# 16. Performance Messung GNN

In [1]:
import numpy as np
import networkx as nx
from tensorflow import keras
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from gml.graph.data_factory import DataFactory
from gml.measure.system_meter import SystemMeter, CpuSystemMeter, MemorySystemMeter, TimeSystemMeter

## 16.1 Messung Graph Erstellung, Training und Link Prediction

In [3]:
# init system meters
meters = [TimeSystemMeter(), MemorySystemMeter(), CpuSystemMeter()]

# init data
labels   = ["Measure A", "Measure B", "Measure C", "Measure D"]
n_graphs = [1000, 2000, 3000, 6000]
measures = []

print("Start performance measures:")
for n_graph in n_graphs:

    # create graph
    for meter in meters:
        meter.start()

    graph = DataFactory().create_graph(n=n_graph, add_dc=True, connected=True)

    # Testdaten
    edgeSplitter = EdgeSplitter(graph.graph)
    test_graph, test_samples, test_labels  = edgeSplitter.train_test_split(p=0.1, method="global", keep_connected=True)

    # Trainingsdaten
    edgeSplitter = EdgeSplitter(test_graph, graph.graph)
    train_graph, train_samples, train_labels = edgeSplitter.train_test_split(p=0.1, method="global", keep_connected=True)

    # Node Features hinzufügen
    node_features="idm"
    eye = np.eye(test_graph.number_of_nodes())                      # Identiy matrix (idm) with size = number of nodes
    idm = {n:eye[i] for i,n in enumerate(test_graph.nodes())}       # Dictionary with node number as key and corresponding idm row as value
    nx.set_node_attributes(test_graph, idm, node_features)          # Assign node features (with name 'idm') to the test_graph nodes

    eye = np.eye(train_graph.number_of_nodes())
    idm = {n:eye[i] for i,n in enumerate(train_graph.nodes())}
    nx.set_node_attributes(train_graph, idm, node_features)

    # Link Generator definieren
    batch_size = 64
    num_samples = [4, 4]

    train_sg   = StellarGraph.from_networkx(train_graph, node_features=node_features)
    train_gen  = GraphSAGELinkGenerator(train_sg, batch_size, num_samples)
    train_flow = train_gen.flow(train_samples, train_labels, shuffle=True, seed=24)

    test_sg    = StellarGraph.from_networkx(test_graph, node_features=node_features)
    test_gen   = GraphSAGELinkGenerator(test_sg, batch_size, num_samples)
    test_flow  = test_gen.flow(test_samples, test_labels, seed=24)

    # Model erstellen
    layer_sizes = [20, 20]
    graphsage = GraphSAGE(
        layer_sizes=layer_sizes,
        generator=train_gen,
        bias=True,
        dropout=0.3)

    x_inp, x_out = graphsage.in_out_tensors()
    prediction   = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.mse,
        metrics=["acc"],
    )

    # Model trainieren
    epochs = 12
    history = model.fit(train_flow, epochs=epochs, validation_data=test_flow)


    # Prediction
    y_pred = np.round(model.predict(test_flow)).flatten()

    # summarize measure results
    measure = {
        "graphs": n_graph,
        "nodes": graph.number_of_nodes(),
        "edges": graph.number_of_edges()
    }

    # stop system meters and add them to the measure
    for meter in meters:
        meter.stop()
        measure[meter.name] = meter.result()

    # save measure
    measures.append(measure)
    print(measure)

Start performance measures:
** Sampled 900 positive and 900 negative edges. **
** Sampled 810 positive and 810 negative edges. **
link_classification: using 'ip' method to combine node embeddings into edge embeddings
Epoch 1/12
26/26 [==============================] - 9s 267ms/step - loss: 0.2573 - acc: 0.4957 - val_loss: 0.2550 - val_acc: 0.5128
Epoch 2/12
26/26 [==============================] - 6s 242ms/step - loss: 0.2344 - acc: 0.6265 - val_loss: 0.2466 - val_acc: 0.5428
Epoch 3/12
26/26 [==============================] - 6s 243ms/step - loss: 0.2153 - acc: 0.7056 - val_loss: 0.2468 - val_acc: 0.5550
Epoch 4/12
26/26 [==============================] - 6s 245ms/step - loss: 0.1984 - acc: 0.7944 - val_loss: 0.2455 - val_acc: 0.5656
Epoch 5/12
26/26 [==============================] - 6s 238ms/step - loss: 0.1808 - acc: 0.8630 - val_loss: 0.2454 - val_acc: 0.5533
Epoch 6/12
26/26 [==============================] - 7s 255ms/step - loss: 0.1714 - acc: 0.8932 - val_loss: 0.2442 - val_acc

## 16.2 Anzeige Resultate

In [4]:
df = SystemMeter.create_df(measures, labels)
print(df)

             Measure A  Measure B  Measure C  Measure D
Graphs         1000.00    2000.00    3000.00    6000.00
Nodes          6003.00   12003.00   18003.00   36003.00
Edges          9002.00   18002.00   27002.00   54002.00
Time [min]        1.55       4.66       9.64      35.40
Memory [mb]  104018.25    4664.26   10450.10   41634.55
Cpu [%]          36.88      49.07      53.12      57.68


---
_The end._